In [4]:
import tensorflow as tf
import urllib
import numpy as np
import zipfile
import pandas as pd

2022-09-09 20:46:23.837905: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-09-09 20:46:23.843387: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-09 20:46:23.843407: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
def download_and_extract_data():
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/certificate/household_power.zip'
    urllib.request.urlretrieve(url, 'household_power.zip')
    with zipfile.ZipFile('household_power.zip', 'r') as zip_ref:
        zip_ref.extractall()

In [6]:
def normalize_series(data, min, max):
    data = data - min
    data = data / max
    return data

In [7]:
def windowed_dataset(series, batch_size, n_past=24, n_future=24, shift=1):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(size=n_past + n_future, shift=shift, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(n_past + n_future))
    ds = ds.map(lambda w: (w[:n_past], w[n_past:]))
    return ds.batch(batch_size).prefetch(1)

In [8]:
download_and_extract_data()
# Reads the dataset from the CSV.
df = pd.read_csv('household_power_consumption.csv', sep=',',
                 infer_datetime_format=True, index_col='datetime', header=0)

# Number of features in the dataset. We use all features as predictors to
# predict all features at future time steps.
N_FEATURES = len(df.columns) # DO NOT CHANGE THIS

# Normalizes the data
data = df.values
data = normalize_series(data, data.min(axis=0), data.max(axis=0))

# Splits the data into training and validation sets.
SPLIT_TIME = int(len(data) * 0.5) # DO NOT CHANGE THIS
x_train = data[:SPLIT_TIME]
x_valid = data[SPLIT_TIME:]

# DO NOT CHANGE THIS CODE
tf.keras.backend.clear_session()
tf.random.set_seed(42)

# DO NOT CHANGE BATCH_SIZE IF YOU ARE USING STATEFUL LSTM/RNN/GRU.
# THE TEST WILL FAIL TO GRADE YOUR SCORE IN SUCH CASES.
# In other cases, it is advised not to change the batch size since it
# might affect your final scores. While setting it to a lower size
# might not do any harm, higher sizes might affect your scores.
BATCH_SIZE = 32  # ADVISED NOT TO CHANGE THIS

# DO NOT CHANGE N_PAST, N_FUTURE, SHIFT. The tests will fail to run
# on the server.
# Number of past time steps based on which future observations should be
# predicted
N_PAST = 24  # DO NOT CHANGE THIS

# Number of future time steps which are to be predicted.
N_FUTURE = 24  # DO NOT CHANGE THIS

# By how many positions the window slides to create a new window
# of observations.
SHIFT = 1  # DO NOT CHANGE THIS

# Code to create windowed train and validation datasets.
train_set = windowed_dataset(series=x_train, batch_size=BATCH_SIZE,
                             n_past=N_PAST, n_future=N_FUTURE,
                             shift=SHIFT)
valid_set = windowed_dataset(series=x_valid, batch_size=BATCH_SIZE,
                             n_past=N_PAST, n_future=N_FUTURE,
                             shift=SHIFT)

2022-09-09 20:46:32.017904: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-09 20:46:32.017930: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-09 20:46:32.017950: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (lscr): /proc/driver/nvidia/version does not exist
2022-09-09 20:46:32.018204: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
x_train.shape

(43200, 7)

In [15]:
def solution_model():
    # Code to define your model.
    model = tf.keras.models.Sequential([
        # tf.keras.layers.Conv1D(filters=64,
        #                        kernel_size=5,
        #                        strides=1,
        #                        activation="relu",
        #                        padding='causal',
        #                        input_shape=(N_PAST, N_FEATURES)),
        
        tf.keras.layers.LSTM(32, return_sequences=True, input_shape=(N_PAST, N_FEATURES)),
        tf.keras.layers.LSTM(32, return_sequences=True),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(8, activation="relu"),

        tf.keras.layers.Dense(N_FEATURES)
    ])
    # Code to train and compile the model
    optimizer = tf.keras.optimizers.Adam() # YOUR CODE HERE
    model.compile(loss=tf.keras.losses.Huber(),
                  optimizer=optimizer,
                  metrics=['mae']
    )
    print(model.summary)
    history = model.fit(train_set,
                        epochs=20,
                        batch_size=BATCH_SIZE,
                        validation_data=valid_set
    )

    return model, history

In [16]:
model, history = solution_model()
model.save("c5q4.h5")

Epoch 1/20
1349/1349 [==============================] - 41s 28ms/step - loss: 0.0127 - mae: 0.0942 - val_loss: 0.0107 - val_mae: 0.0794
Epoch 2/20
1349/1349 [==============================] - 37s 27ms/step - loss: 0.0105 - mae: 0.0800 - val_loss: 0.0101 - val_mae: 0.0746
Epoch 3/20
1349/1349 [==============================] - 37s 28ms/step - loss: 0.0101 - mae: 0.0756 - val_loss: 0.0097 - val_mae: 0.0690
Epoch 4/20
1349/1349 [==============================] - 38s 28ms/step - loss: 0.0099 - mae: 0.0735 - val_loss: 0.0096 - val_mae: 0.0680
Epoch 5/20
1349/1349 [==============================] - 38s 28ms/step - loss: 0.0098 - mae: 0.0726 - val_loss: 0.0096 - val_mae: 0.0684
Epoch 6/20
1349/1349 [==============================] - 37s 28ms/step - loss: 0.0098 - mae: 0.0723 - val_loss: 0.0095 - val_mae: 0.0680
Epoch 7/20
1349/1349 [==============================] - 39s 29ms/step - loss: 0.0097 - mae: 0.0718 - val_loss: 0.0095 - val_mae: 0.0679
Epoch 8/20
1349/1349 [==========================

In [ ]:
# THIS CODE IS USED IN THE TESTER FOR FORECASTING. IF YOU WANT TO TEST YOUR MODEL
# BEFORE UPLOADING YOU CAN DO IT WITH THIS
#def mae(y_true, y_pred):
#    return np.mean(abs(y_true.ravel() - y_pred.ravel()))
#
#
#def model_forecast(model, series, window_size, batch_size):
#    ds = tf.data.Dataset.from_tensor_slices(series)
#    ds = ds.window(window_size, shift=1, drop_remainder=True)
#    ds = ds.flat_map(lambda w: w.batch(window_size))
#    ds = ds.batch(batch_size, drop_remainder=True).prefetch(1)
#    forecast = model.predict(ds)
#    return forecast
#

# PASS THE NORMALIZED data IN THE FOLLOWING CODE

#rnn_forecast = model_forecast(model, data, N_PAST, BATCH_SIZE)
#rnn_forecast = rnn_forecast[SPLIT_TIME - N_PAST:-1, 0, :]

#x_valid = x_valid[:rnn_forecast.shape[0]]
#result = mae(x_valid, rnn_forecast)